### Q2: Implement the DLSPnP-PE algorithm according to the paper [Zheng and Kneip,2016] and test it on the provided outlier-free data. Varying the number of points from 4 to 15, please report and analyse the following metrics

•Reprojection Errors

•Absolute Rotation Error in degree

•Relative Translation Error in percentage

•Relative Focal Length Error in percentage

### Q3: Implement RANSAC with DLSPnP-PE as the minimal solver on the corrupted data provided and present the aforementioned metrics too.


In [19]:
import sympy as sp
import numpy as np

K_gt = np.array([[800,0,400],
             [0,800,320],
             [0,0,1]])
R_gt = np.array([[0.97021030670802,-0.103777351275851,0.218911448123891],
             [0.0955355748687254,0.994271247550756,0.0479337065994893],
             [-0.222631791737013,-0.0255919451396816,0.974566640949676]])
t_gt = np.array([[0.152958693028076],
                [-1.58419349813393],
                [1.25940311352436]])

data = '/home/daizj/Homeworks/gv/Homework2/data/data4/'
U = np.loadtxt(data + 'u.txt')
X = np.loadtxt(data + 'x.txt')
U[:,0]-=400
U[:,1]-=320
U = U[:5,:]
X = X[:5,:]
print("U", U.shape)
print("X", X.shape)

U (5, 3)
X (5, 3)


In [20]:
from sympy import *
x,y,b,c,k = sp.symbols('x y b c k')

r1,r2,r3 = sp.symbols('r_1 r_2 r_3')

t1,t2,t3 = sp.symbols('t_1 t_2 t_3')

lambi,ui,vi,xi,yi,zi = sp.symbols('lambda_i u_i v_i x_i y_i z_i')

# Rbc = sp.symbols('R(b,c)')
k = 1#expand(1 + b*b + c*c)
Rbc = (1/k) * Matrix([[1+b*b-c*c, 2*b*c, 2*c],
                    [2*b*c, 1-b*b+c*c, -2*b],
                    [-2*c, 2*b, 1-b*b-c*c]])
KRq_theta = Matrix([[x, -y, 0],
                    [y, x, 0],
                    [0, 0, 1]])

KR = KRq_theta * Rbc

r1, r2, r3 = KR[0,:].T, KR[1,:].T, KR[2,:].T

ui,vi = U[:,0],U[:,1]
ui = ui.reshape(-1,1)
vi = vi.reshape(-1,1)
Xi = X
Ui = U

uiXi = np.multiply(ui,Xi)
viXi = np.multiply(vi,Xi)

uiXi_decentral = uiXi - np.average(uiXi, axis = 0)
viXi_decentral = viXi - np.average(viXi, axis = 0)

ui_decentral = ui - np.average(ui, axis = 0)
vi_decentral = vi - np.average(vi, axis = 0)
Xi_decentral = Xi - np.average(Xi, axis = 0)

s = np.multiply(ui_decentral,ui_decentral) + np.multiply(vi_decentral,vi_decentral) 
s = np.sum(s)
print("s",s)
temp = np.multiply(ui_decentral, uiXi_decentral) + np.multiply(vi_decentral, viXi_decentral)
f1 = np.sum(np.multiply(ui_decentral,Xi_decentral),axis = 0)
f2 = np.sum(np.multiply(vi_decentral,Xi_decentral),axis = 0)
f3 = np.sum(temp,axis = 0)
t3 = -(r3.T @ f3.T - r1.T @ f1.T - r2.T @ f2.T)/s

print(f1)
print(f2)
print(f3)
print(t3)

s 119954.54520269898
[718.7992664  252.83869755 150.4351745 ]
[274.56109751 486.99881515 309.33728777]
[-445776.33060189   94089.48918081  754766.23303087]
[6.29210199376329*b**2 + 0.00879332740860363*b*c*x - 0.00386480480340246*b*c*y - 0.00515757510062339*b*x + 0.00250820299046025*b*y - 1.56875238069243*b + 6.29210199376329*c**2 + 0.00250820299046025*c*x + 0.00515757510062339*c*y - 7.43242083655301*c + 0.00405986129438342*x*(-b**2 + c**2 + 1) + 0.00599226369608465*x*(b**2 - c**2 + 1) - 0.0021077875550247*y*(-b**2 + c**2 + 1) + 0.00228887614927711*y*(b**2 - c**2 + 1) - 6.29210199376329]


In [21]:
# p3i = uiXi_decentral - ui_decentral * (f3/s)
# p1i = Xi_decentral - ui_decentral * (f1/s)
# p2i = - ui_decentral * (f2/s)

# q3i = viXi_decentral - vi_decentral * (f3/s)
# q2i = Xi_decentral - vi_decentral * (f2/s)
# q1i = - vi_decentral * (f1/s)

# eq1 = r3.T * p3i.T - r1.T * p1i.T - r2.T * p2i.T
# eq2 = r3.T * q3i.T - r1.T * q1i.T - r2.T * q2i.T

eq1 = r3.T * uiXi_decentral.T + ui_decentral.T * t3 - r1.T * Xi_decentral.T
eq2 = r3.T * viXi_decentral.T + vi_decentral.T * t3 - r2.T * Xi_decentral.T

ax = eq1[0].coeff('x')
ay = eq1[0].coeff('y')
ebc = simplify(eq1[0] - ax * x - ay * y)
Mbc = Matrix([[ax,ay]])
print(ebc)
print(Mbc)

-65.6266501091959*b**2 + 38.3901869993959*b - 65.6266501091959*c**2 + 373.187887702616*c + 65.6266501091959
Matrix([[-0.697843029013381*b**2 + 2.07905480596047*b*c - 0.262949069311432*b + 0.697843029013381*c**2 + 1.98707238820994*c - 0.283874554269273, 1.03952740298024*b**2 + 1.39568605802676*b*c + 1.98707238820994*b - 1.03952740298024*c**2 + 0.262949069311432*c - 0.806139486574014]])


In [22]:
Mbcs = []
ebcs = []
n = ui.shape[0]
print("num", n)
for i in range(n):
    ax = eq1[i].coeff('x')
    ay = eq1[i].coeff('y')
    # ebc = simplify(eq1[i] - ax * x - ay * y)
    ebc = -eq1[i].subs('x' ,0).subs('y',0)
    Mbc = Matrix([[ax,ay]])
    ebcs.append(ebc)
    Mbcs.append(Mbc)
    
    ax = eq2[i].coeff('x')
    ay = eq2[i].coeff('y')
    # ebc = simplify(eq2[i] - ax * x - ay * y)
    ebc = -eq2[i].subs('x' ,0).subs('y',0)
    Mbc = Matrix([[ax,ay]])
    ebcs.append(ebc)
    Mbcs.append(Mbc)
Mbcs = Matrix(Mbcs)
ebcs = Matrix(ebcs)
# ebcs

num 5


In [23]:
Mbcs = Matrix(Mbcs)
ebcs = Matrix(ebcs)

MtM = Mbcs.T * Mbcs
MtM_adj = MtM.adjugate()
MtM_det = det(MtM)
I = eye(2 * n)

phi = - ebcs.T * (Mbcs * (MtM_adj) * Mbcs.T - MtM_det * I) * ebcs
phi = phi[0,0].expand()

phi_b = phi.diff(b)
phi_c = phi.diff(c)
print(str(phi_b).replace("**", "^"))

165254017.998395*b^11 + 8.50043847250031e-9*b^10*c - 8206810.12288615*b^10 + 826270089.991977*b^9*c^2 - 265298395.950384*b^9*c - 188212626.26154*b^9 - 3.4514300134693e-9*b^8*c^3 - 33573314.139079*b^8*c^2 + 828074696.572995*b^8*c - 370313913.766225*b^8 + 1652540179.98395*b^7*c^4 - 1061193583.80153*b^7*c^3 + 264655200.766125*b^7*c^2 + 889113278.880223*b^7*c + 255081988.151428*b^7 - 8.09381550307733e-9*b^6*c^5 - 52225155.3274557*b^6*c^4 + 2576232389.33821*b^6*c^3 - 1182386040.43199*b^6*c^2 + 1099707772.13737*b^6*c + 433591854.963847*b^6 + 1652540179.98395*b^5*c^6 - 1591790375.7023*b^5*c^5 + 1923241359.86761*b^5*c^4 + 1919085200.88671*b^5*c^3 + 2679328026.12658*b^5*c^2 + 38233950.6233447*b^5*c + 414690804.077112*b^5 + 5.78129678791238e-9*b^4*c^7 - 37303682.3767535*b^4*c^6 + 2760248988.57665*b^4*c^5 - 1299304659.80017*b^4*c^4 + 3780700359.20858*b^4*c^3 + 3018542219.05276*b^4*c^2 + 977716718.229706*b^4*c + 269229979.905072*b^4 + 826270089.991976*b^3*c^8 - 1061193583.80153*b^3*c^7 + 229966661

In [24]:
print(str(phi_c).replace("**", "^"))

7.72767133863665e-10*b^11 + 165254017.998395*b^10*c - 26529839.5950384*b^10 - 1.15047667115643e-9*b^9*c^2 - 7460736.47535088*b^9*c + 92008299.6192217*b^9 + 826270089.991977*b^8*c^3 - 397947593.925575*b^8*c^2 + 66163800.1915312*b^8*c + 111139159.860028*b^8 - 5.78129678791238e-9*b^7*c^4 - 29842945.9014032*b^7*c^3 + 1104099595.43066*b^7*c^2 - 337824582.980568*b^7*c + 157101110.305339*b^7 + 1652540179.98395*b^6*c^5 - 1326491979.75192*b^6*c^4 + 1282160906.57841*b^6*c^3 + 959542600.443354*b^6*c^2 + 893109342.042193*b^6*c + 6372325.10389078*b^6 + 8.09381550307733e-9*b^5*c^6 - 44764418.8521042*b^5*c^5 + 2760248988.57665*b^5*c^4 - 1039443727.84014*b^5*c^3 + 2268420215.52515*b^5*c^2 + 1207416887.6211*b^5*c + 195543343.645941*b^5 + 1652540179.98395*b^4*c^7 - 1857088771.65268*b^4*c^6 + 3449499918.58604*b^4*c^5 + 1463538206.41594*b^4*c^4 + 3401626446.12736*b^4*c^3 + 474228853.251069*b^4*c^2 + 1413138906.89996*b^4*c + 110358837.012871*b^4 + 3.4514300134693e-9*b^3*c^8 - 29842945.9014026*b^3*c^7 + 257

In [36]:
with open("finite_solutions", "r") as file:
    lines = [line.strip() for line in file if line.strip()]
    
used_solu = []
for i in range(int(lines[0])):
    line1 = lines[i * 2 + 1].split(" ")
    line2 = lines[i * 2 + 2].split(" ")
    
    b, b_ver = map(float, line1)
    c, c_ver = map(float, line2)
    
    if abs(b_ver) < 1e-10 and abs(c_ver) < 1e-10:
        used_solu.append((b, c))

In [54]:
min_err = 999999
R_result, t_result, f_result=0,0,0
for b, c in used_solu:
    Mbc = np.array(Mbcs.subs({"b": b, "c": c}), dtype=np.float64)
    ebc = np.array(ebcs.subs({"b": b, "c": c}), dtype=np.float64)
    X = np.linalg.lstsq(Mbc, ebc, rcond=None)[0]
    x, y = X[0, 0], X[1, 0]
    f = np.sqrt(x**2 + y**2)

    k = 1 + b**2 + c**2
    R = (KRq_theta @ Rbc).subs({"x": x, "y": y, "b": b, "c": c})
    R = np.array(R / k, dtype=np.float64)
    R = np.diag([1 / f, 1 / f, 1]) @ R

    t3_value = np.array(t3[0].subs({"b": b, "c": c, "x": x, "y": y}), dtype=np.float64)
    r3_value = np.array(r3.subs({"b": b, "c": c, "x": x, "y": y}), dtype=np.float64)

    lambi_value = np.dot(r3_value.T, Xi[0].reshape(3, 1)) + t3_value
    r1_value = np.array(r1.subs({"b": b, "c": c, "x": x, "y": y}), dtype=np.float64)
    r2_value = np.array(r2.subs({"b": b, "c": c, "x": x, "y": y}), dtype=np.float64)
    t1_value = lambi_value * ui[0] - np.dot(r1_value.T, Xi[0].reshape(3, 1))
    t2_value = lambi_value * vi[0] - np.dot(r2_value.T, Xi[0].reshape(3, 1))

    t1_value = np.array(t1_value).astype(np.float64).reshape(-1)
    t2_value = np.array(t2_value).astype(np.float64).reshape(-1)
    t3_value = np.array(t3_value).astype(np.float64).reshape(-1)
    
    t = ((1 / k)) * np.diag([1 / f, 1 / f, 1]) @ np.array([t1_value, t2_value, t3_value]).reshape(3, 1)

    K = np.diag([f, f, 1])
    Xp = np.dot(Xi, R.T) + t.reshape(1, 3)
    Ui_rep = np.dot(K, Xp.T).T
    Ui_rep = Ui_rep / Ui_rep[:, 2:3]

    rep_err = np.linalg.norm(Ui_rep - Ui, axis=1)
        
    if np.average(rep_err) < min_err:
        min_err = np.average(rep_err)
        R_result = R
        t_result = t
        f_result = f

In [55]:
# reprojection error
print("reprojection error:", min_err)

reprojection error: 1.480271082406342e-13


In [56]:
# Absolute Rotation Error in degree
r_err = np.linalg.norm(best_R @ R_gt.T - np.eye(3), ord='fro')
print("Absolute Rotation Error:", r_err)

Absolute Rotation Error: 3.1109265818524668e-15


In [57]:
# Relative Translation Error in percentage
t_err = np.linalg.norm(best_t.reshape(-1) - t_gt.reshape(-1)) / np.linalg.norm(t)
print("Relative Translation Error:",t_err)

Relative Translation Error: 3.6529617510443754e-14


In [58]:
# Relative Focal Length Error in percentage
f_err = np.abs(best_f - K_gt[0,0])/K_gt[0,0]
print("focal error:", f_err)

focal error: 8.668621376273222e-15
